In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from icecream import ic
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from xgboost import XGBClassifier

import random
import numpy as np
import torch

from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, confusion_matrix

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, confusion_matrix
import numpy as np
import time

from torch.utils.data import Dataset, DataLoader, random_split



def set_seed(seed_value=42):
    """Set seed for reproducibility across numpy, torch, and random."""
    random.seed(seed_value)  # Python's built-in random module
    np.random.seed(seed_value)  # NumPy random seed

    torch.manual_seed(seed_value)  # PyTorch random seed for CPU
    torch.cuda.manual_seed(seed_value)  # PyTorch random seed for CUDA
    torch.cuda.manual_seed_all(seed_value)  # PyTorch seed for all GPUs

    torch.backends.cudnn.deterministic = True  # Ensures deterministic behavior in cuDNN
    torch.backends.cudnn.benchmark = False  # Disables cuDNN auto-optimization

# Usage
set_seed(42)  # Call this function before running models or data splits to ensure reproducibility

device = 'cuda' if torch.cuda.is_available() else 'cpu'


/mnt/storageG1/lwang/miniconda3/envs/new-ml/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


### data

In [6]:
# df = pd.read_csv('all_seq722.csv')

# df = df[~df["Sequences"].str.contains('-')]
# df['Sequences'] = df['Sequences'].str.upper()
# max_length = df['Sequences'].str.len().max()
# print(max_length)
# df['Sequences'] = df['Sequences'].apply(lambda x: x.ljust(max_length, 'X')

df = pd.read_csv('../data/all_seq702.csv')
max_length = df['Sequences'].str.len().max()
print(max_length)
df['Sequences'] = df['Sequences'].apply(lambda x: x.ljust(max_length, 'X'))

unique_letters = set(''.join(df["Sequences"]))
print(unique_letters)
print(len(unique_letters))
amino_acids = set("ACDEFGHIKLMNPQRSTVWY")
non_standard_amino_acids = unique_letters - amino_acids
print(non_standard_amino_acids)
b_count = df["Sequences"].str.count('B').sum()
print(f"Number of 'B' values: {b_count}")
# manually replaced one of the B with D and the other with N

128
{'N', 'M', 'T', 'F', 'I', 'C', 'K', 'G', 'V', 'W', 'D', 'E', 'X', 'S', 'H', 'Y', 'A', 'P', 'L', 'R', 'Q'}
21
{'X'}
Number of 'B' values: 0


# Classification

In [7]:
# Re-import necessary libraries after execution state reset
# Define One-Hot Encoding Function for DNA Sequences in PyTorch
def one_hot_torch(seq: str, dtype=torch.float32):
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    seq_bytes = torch.ByteTensor(list(bytes(seq, "utf-8")))
    aa_bytes = torch.ByteTensor(list(bytes(amino_acids, "utf-8")))
    arr = torch.zeros(len(amino_acids), len(seq_bytes), dtype=dtype)
    for i, aa in enumerate(aa_bytes):
        arr[i, seq_bytes == aa] = 1
    return arr

# Define custom dataset class with transformation

# Updating the Dataset class with the OneHotEncoder function at the end
class SequenceDataset(Dataset):
    def __init__(self, sequences, labels, one_hot_dtype=torch.float32):
        self.sequences = sequences  # Raw sequences
        self.labels = labels  # Labels
        self.one_hot_dtype = one_hot_dtype  # Data type for one-hot encoding

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seqs_comb = self.sequences.iloc[idx]  # Get sequence
        amp_label = self.labels.iloc[idx]    # Get corresponding label
        # Apply one-hot encoding transformation at the end
        return one_hot_torch(seqs_comb, dtype=self.one_hot_dtype), torch.tensor(amp_label, dtype=torch.float32)
    
# Convert dataset into PyTorch Dataset

X = df["Sequences"]
y = df["AMP"]
# Split into train (70%), validation (15%), test (15%)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y
)

# Step 2: Split train+val into train and val (stratified)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.15, random_state=42, stratify=y_train_val
)  # 0.1765 to maintain 15% of original dataset

# Convert back to PyTorch datasets
train_dataset = SequenceDataset(X_train, y_train)
val_dataset = SequenceDataset(X_val, y_val)
test_dataset = SequenceDataset(X_test, y_test)

# Define DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Display dataset sizes
dataset_sizes = {
    "Train": len(train_dataset),
    "Validation": len(val_dataset),
    "Test": len(test_dataset)
}
# for x,y in train_loader:
#     print(x)
#     print(y)
#     break

In [10]:

X_train

27     IRMRIRVLLXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...
194    WALVNXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...
201    WLLVXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...
41     RRYHWRIYIXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...
302    INLKALAALAKKILXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...
                             ...                        
652    GPQQQHRLXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...
182    RQIKIWFQNRRMKWKKXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...
670    VHWSAEEKQLXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...
21     HQFRFRFRVRRKXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...
601    WREMSVWXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...
Name: Sequences, Length: 508, dtype: object

In [ ]:
# Function to calculate specificity
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

# Training and Evaluation Function
def train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, tensorboard_,num_epochs=100):
    writer = SummaryWriter(tensorboard_)
    for epoch in range(num_epochs):
        model.train()
        train_losses = []
        for sequences, labels in train_loader:
            optimizer.zero_grad()
            ic(sequences.shape)
            outputs = model(sequences)
            loss = criterion(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        
        avg_train_loss = np.mean(train_losses)
        writer.add_scalar('Loss/Train', avg_train_loss, epoch)

        # Evaluation
        model.eval()
        all_labels = []
        all_preds = []
        val_losses = []
        all_probs = []
        with torch.no_grad():
            for sequences, labels in val_loader:
                outputs = model(sequences)
                loss = criterion(outputs, labels.unsqueeze(1))
                val_losses.append(loss.item())
                preds = (outputs > 0.5).float()
                all_probs.extend(outputs.numpy())
                all_labels.extend(labels.numpy())
                all_preds.extend(preds.numpy())
                
            avg_val_loss = np.mean(val_losses)
            writer.add_scalar('Loss/Val', avg_val_loss, epoch)

        all_labels = np.array(all_labels)
        all_preds = np.array(all_preds)
        accuracy = accuracy_score(all_labels, all_preds)
        auc = roc_auc_score(all_labels, all_probs)
        sensitivity = recall_score(all_labels, all_preds)
        specificity = specificity_score(all_labels, all_preds)

        writer.add_scalar('Metrics/Accuracy', accuracy, epoch)
        writer.add_scalar('Metrics/AUC', auc, epoch)
        writer.add_scalar('Metrics/Sensitivity', sensitivity, epoch)
        writer.add_scalar('Metrics/Specificity', specificity, epoch)
        if epoch % 5 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, TrainLoss: {avg_train_loss:.4f}, ValLoss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}, AUC: {auc:.4f}, Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}')

    writer.close()
    


def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    all_labels = []
    all_preds = []
    all_probs = []
    
    with torch.no_grad():  # Disable gradient computation
        for sequences, labels in test_loader:
            outputs = model(sequences)
            preds = (outputs > 0.5).float()
            all_probs.extend(outputs.numpy())
            all_labels.extend(labels.numpy())
            all_preds.extend(preds.numpy())
    
    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)
    
    accuracy = accuracy_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    sensitivity = recall_score(all_labels, all_preds)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
    specificity = tn / (tn + fp)
    
    print(accuracy)
    print(auc)
    print(f'Test Accuracy: {accuracy:.4f}')
    print(f'Test AUC: {auc:.4f}')
    print(f'Test Sensitivity (Recall): {sensitivity:.4f}')
    print(f'Test Specificity: {specificity:.4f}')
    
    return accuracy, auc, sensitivity, specificity


## basic lstm

In [9]:


ic.disable()
# Define the Basic LSTM Model
class BasicLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(BasicLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        ic(h_n.shape)
        ic(h_n[-1].shape)
        ic(h_n[-1])
        out = self.fc(h_n[-1])
        return self.sigmoid(out)


# Hyperparameters
input_size = 128       
hidden_size = 128    # Number of features in the hidden state
num_layers = 2       # Number of stacked LSTM layers
output_size = 1      # Binary classification (AMP or not)
batch_size = 16
num_epochs = 21
learning_rate = 0.001
tensorboard_ = f"runs/basic_lstm/experiment_{time.strftime('%Y-%m-%d_%H-%M-%S')}"


# Model, Loss, Optimizer
model = BasicLSTM(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train and Evaluate
train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, tensorboard_,num_epochs)

evaluate_model(model, test_loader)



Epoch 1/21, TrainLoss: 0.6911, ValLoss: 0.6871, Accuracy: 0.5000, AUC: 0.6602, Sensitivity: 0.0000, Specificity: 1.0000
Epoch 6/21, TrainLoss: 0.5061, ValLoss: 0.5599, Accuracy: 0.7222, AUC: 0.7995, Sensitivity: 0.6667, Specificity: 0.7778
Epoch 11/21, TrainLoss: 0.3502, ValLoss: 0.5525, Accuracy: 0.7444, AUC: 0.8311, Sensitivity: 0.6444, Specificity: 0.8444
Epoch 16/21, TrainLoss: 0.2752, ValLoss: 0.7030, Accuracy: 0.7778, AUC: 0.8049, Sensitivity: 0.7333, Specificity: 0.8222
Epoch 21/21, TrainLoss: 0.2119, ValLoss: 0.7577, Accuracy: 0.7222, AUC: 0.8188, Sensitivity: 0.7556, Specificity: 0.6889
0.6981132075471698
0.7985048059807761
Test Accuracy: 0.6981
Test AUC: 0.7985
Test Sensitivity (Recall): 0.7925
Test Specificity: 0.6038


(0.6981132075471698,
 0.7985048059807761,
 0.7924528301886793,
 0.6037735849056604)

## lstm with attention

In [294]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ScaledDotProductAttention(nn.Module):
    def __init__(self, hidden_size):
        super(ScaledDotProductAttention, self).__init__()
        self.scale_factor = torch.sqrt(torch.tensor(hidden_size, dtype=torch.float32))
    
    def forward(self, query, key, value):
        attention_scores = torch.bmm(query, key.transpose(1, 2))  # [batch, seq_len, seq_len]
        attention_scores = attention_scores / self.scale_factor
        attention_weights = F.softmax(attention_scores, dim=-1)  # Normalize scores
        context_vector = torch.bmm(attention_weights, value)  # Weighted sum
        return context_vector, attention_weights
    
class LSTM_ScaledDotAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM_ScaledDotAttention, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.attention = ScaledDotProductAttention(hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # LSTM output shape: [batch_size, seq_len, hidden_size]
        
        # Using last hidden state as Query
        query = lstm_out[:, -1, :].unsqueeze(1)  # [batch_size, 1, hidden_size]

        # Key and Value are the full sequence outputs
        key = lstm_out  # [batch_size, seq_len, hidden_size]
        value = lstm_out

        context_vector, attention_weights = self.attention(query, key, value)
        context_vector = context_vector.squeeze(1)  # [batch_size, hidden_size]

        out = self.fc(context_vector)  # [batch_size, output_size]
        return self.sigmoid(out)


In [295]:
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, confusion_matrix
import numpy as np

import torch.optim as optim
import time

# Hyperparameters
input_size = 128  # One-hot encoding size or embedding dimension
hidden_size = 128  # LSTM hidden state size
num_layers = 2  # Stacked LSTM layers
output_size = 1  # Binary classification
batch_size = 16
num_epochs = 12
learning_rate = 0.001
tensorboard_log_dir = f"runs/lstm_attention/experiment_{time.strftime('%Y-%m-%d_%H-%M-%S')}"

# Model, Loss, Optimizer
model = LSTM_ScaledDotAttention(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train and Evaluate
train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, tensorboard_log_dir, num_epochs)
evaluate_model(model, test_loader)


Epoch 1/12, TrainLoss: 0.6937, ValLoss: 0.6900, Accuracy: 0.5000, AUC: 0.6568, Sensitivity: 0.0000, Specificity: 1.0000
Epoch 6/12, TrainLoss: 0.5912, ValLoss: 0.5631, Accuracy: 0.7222, AUC: 0.7921, Sensitivity: 0.5556, Specificity: 0.8889
Epoch 11/12, TrainLoss: 0.5316, ValLoss: 0.5962, Accuracy: 0.7222, AUC: 0.7960, Sensitivity: 0.5111, Specificity: 0.9333
0.6320754716981132
0.7080811676753294
Test Accuracy: 0.6321
Test AUC: 0.7081
Test Sensitivity (Recall): 0.4151
Test Specificity: 0.8491


(0.6320754716981132,
 0.7080811676753294,
 0.41509433962264153,
 0.8490566037735849)

In [ ]:
# Evaluate


0.7830188679245284
0.8305446778212888
Test Accuracy: 0.7830
Test AUC: 0.8305
Test Sensitivity (Recall): 0.8302
Test Specificity: 0.7358


(0.7830188679245284,
 0.8305446778212888,
 0.8301886792452831,
 0.7358490566037735)

## CNN-LSTM with attention

In [297]:

import torch
import torch.nn as nn
import torch.nn.functional as F
ic.disable()
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attention_weights = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, lstm_output):
        # lstm_output: [batch_size, seq_len, hidden_dim]
        attention_scores = self.attention_weights(lstm_output)  # [batch_size, seq_len, 1]
        attention_scores = attention_scores.squeeze(-1)  # [batch_size, seq_len]
        attention_weights = F.softmax(attention_scores, dim=1)  # [batch_size, seq_len]
        context_vector = torch.bmm(attention_weights.unsqueeze(1), lstm_output)  # [batch_size, 1, hidden_dim]
        context_vector = context_vector.squeeze(1)  # [batch_size, hidden_dim]
        return context_vector, attention_weights

class CNN_LSTM_Attention(nn.Module):
    def __init__(self, input_channels, cnn_output_dim, lstm_hidden_dim, lstm_layers, output_dim):
        super(CNN_LSTM_Attention, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(input_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        self.lstm = nn.LSTM(cnn_output_dim, lstm_hidden_dim, lstm_layers, batch_first=True)
        self.attention = Attention(lstm_hidden_dim)
        self.fc = nn.Linear(lstm_hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: [batch_size, seq_len, input_channels]
        batch_size, seq_len, _ = x.size()
        ic(x.shape)
        x = x.view(batch_size * seq_len, -1, x.size(2))  # [batch_size * seq_len, input_channels, feature_dim]
        ic(x.shape)
        cnn_out = self.cnn(x)  # [batch_size * seq_len, 128, feature_dim/4]
        cnn_out = cnn_out.view(batch_size, seq_len, -1)  # [batch_size, seq_len, cnn_output_dim]
        lstm_out, _ = self.lstm(cnn_out)  # [batch_size, seq_len, lstm_hidden_dim]
        context_vector, attention_weights = self.attention(lstm_out)  # [batch_size, lstm_hidden_dim]
        out = self.fc(context_vector)  # [batch_size, output_dim]
        return self.sigmoid(out)



In [296]:
# Hyperparameters
input_channels = 1  # Number of input channels
cnn_output_dim = 128 * (input_size // 4)  # Adjust based on CNN architecture
lstm_hidden_dim = 128
lstm_layers = 2
output_dim = 1  # Binary classification
batch_size = 16
num_epochs = 12
learning_rate = 0.001
tensorboard_log_dir = f"runs/cnn_lstm_attention/experiment_{time.strftime('%Y-%m-%d_%H-%M-%S')}"

# Model, Loss, Optimizer
model = CNN_LSTM_Attention(input_channels, cnn_output_dim, lstm_hidden_dim, lstm_layers, output_dim)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train and Evaluate
train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, tensorboard_log_dir, num_epochs)

# Evaluate on Test Set
evaluate_model(model, test_loader)


Epoch 1/12, TrainLoss: 0.6967, ValLoss: 0.6915, Accuracy: 0.5000, AUC: 0.6731, Sensitivity: 0.0000, Specificity: 1.0000
Epoch 6/12, TrainLoss: 0.5147, ValLoss: 0.4816, Accuracy: 0.7222, AUC: 0.8583, Sensitivity: 0.5778, Specificity: 0.8667
Epoch 11/12, TrainLoss: 0.3625, ValLoss: 0.4362, Accuracy: 0.8111, AUC: 0.8765, Sensitivity: 0.7333, Specificity: 0.8889
0.7452830188679245
0.8227127091491635
Test Accuracy: 0.7453
Test AUC: 0.8227
Test Sensitivity (Recall): 0.7358
Test Specificity: 0.7547


(0.7452830188679245,
 0.8227127091491635,
 0.7358490566037735,
 0.7547169811320755)

## bidirectional LSTM

In [191]:
import torch
import torch.nn as nn

class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)  # Multiply hidden_size by 2 for bidirectional
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        _, (h_n, _) = self.lstm(x)  # LSTM output: (batch_size, seq_len, hidden_size * 2)
        # Concatenate the last hidden states from both directions
        out = self.fc(torch.cat((h_n[-2], h_n[-1]), dim=1))  
        return self.sigmoid(out)


In [192]:
import torch.optim as optim
import time

# Hyperparameters
input_size = 128  # Number of features per time step (e.g., One-hot encoding size)
hidden_size = 128  # Number of LSTM units
num_layers = 2  # Stacked LSTM layers
output_size = 1  # Binary classification (AMP or not)
batch_size = 16
num_epochs = 17
learning_rate = 0.001
tensorboard_log_dir = f"runs/bilstm/experiment_{time.strftime('%Y-%m-%d_%H-%M-%S')}"

# Model, Loss, Optimizer
model = BiLSTM(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train and Evaluate
train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, tensorboard_log_dir, num_epochs)
evaluate_model(model, test_loader)


Epoch 1/17, TrainLoss: 0.6942, ValLoss: 0.6882, Accuracy: 0.5333, AUC: 0.7600, Sensitivity: 0.9778, Specificity: 0.0889
Epoch 6/17, TrainLoss: 0.3690, ValLoss: 0.6131, Accuracy: 0.7667, AUC: 0.8173, Sensitivity: 0.7556, Specificity: 0.7778
Epoch 11/17, TrainLoss: 0.2692, ValLoss: 0.6020, Accuracy: 0.7778, AUC: 0.8514, Sensitivity: 0.7111, Specificity: 0.8444
Epoch 16/17, TrainLoss: 0.2030, ValLoss: 0.7112, Accuracy: 0.7889, AUC: 0.8400, Sensitivity: 0.7778, Specificity: 0.8000
0.7830188679245284
0.850124599501602
Test Accuracy: 0.7830
Test AUC: 0.8501
Test Sensitivity (Recall): 0.8113
Test Specificity: 0.7547


(0.7830188679245284, 0.850124599501602, 0.8113207547169812, 0.7547169811320755)

## biLSTM with attention

In [298]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ScaledDotProductAttention(nn.Module):
    def __init__(self, hidden_size):
        super(ScaledDotProductAttention, self).__init__()
        self.scale_factor = torch.sqrt(torch.tensor(hidden_size, dtype=torch.float32))
    
    def forward(self, query, key, value):
        attention_scores = torch.bmm(query, key.transpose(1, 2))  # [batch, seq_len, seq_len]
        attention_scores = attention_scores / self.scale_factor
        attention_weights = F.softmax(attention_scores, dim=-1)  # Normalize scores
        context_vector = torch.bmm(attention_weights, value)  # Weighted sum
        return context_vector, attention_weights
    
class BiLSTM_ScaledDotAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(BiLSTM_ScaledDotAttention, self).__init__()
        self.bilstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.attention = ScaledDotProductAttention(hidden_size * 2)  # Bidirectional -> 2x hidden size
        self.fc = nn.Linear(hidden_size * 2, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        lstm_out, _ = self.bilstm(x)  # LSTM output shape: [batch_size, seq_len, hidden_size * 2]
        
        # Using last hidden state as Query
        query = lstm_out[:, -1, :].unsqueeze(1)  # [batch_size, 1, hidden_size * 2]

        # Key and Value are the full sequence outputs
        key = lstm_out  # [batch_size, seq_len, hidden_size * 2]
        value = lstm_out

        context_vector, attention_weights = self.attention(query, key, value)
        context_vector = context_vector.squeeze(1)  # [batch_size, hidden_size * 2]

        out = self.fc(context_vector)  # [batch_size, output_size]
        return self.sigmoid(out)


In [299]:
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, confusion_matrix
import numpy as np

# Hyperparameters
input_size = 128  # One-hot encoding size or embedding dimension
hidden_size = 128  # LSTM hidden state size
num_layers = 2  # Stacked BiLSTM layers
output_size = 1  # Binary classification
batch_size = 16
num_epochs = 22
learning_rate = 0.0001
tensorboard_log_dir = f"runs/bilstm_attention/experiment_{time.strftime('%Y-%m-%d_%H-%M-%S')}"

# Model, Loss, Optimizer
model = BiLSTM_ScaledDotAttention(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train and Evaluate
train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, tensorboard_log_dir, num_epochs)


evaluate_model(model, test_loader)


Epoch 1/22, TrainLoss: 0.6938, ValLoss: 0.6931, Accuracy: 0.5000, AUC: 0.6563, Sensitivity: 1.0000, Specificity: 0.0000
Epoch 6/22, TrainLoss: 0.6840, ValLoss: 0.6767, Accuracy: 0.6889, AUC: 0.6454, Sensitivity: 0.4889, Specificity: 0.8889
Epoch 11/22, TrainLoss: 0.6453, ValLoss: 0.6568, Accuracy: 0.5444, AUC: 0.6741, Sensitivity: 0.6667, Specificity: 0.4222
Epoch 16/22, TrainLoss: 0.5764, ValLoss: 0.5407, Accuracy: 0.7556, AUC: 0.8188, Sensitivity: 0.6444, Specificity: 0.8667
Epoch 21/22, TrainLoss: 0.5198, ValLoss: 0.4896, Accuracy: 0.7556, AUC: 0.8326, Sensitivity: 0.7111, Specificity: 0.8000
0.7169811320754716
0.769312922748309
Test Accuracy: 0.7170
Test AUC: 0.7693
Test Sensitivity (Recall): 0.8113
Test Specificity: 0.6226


(0.7169811320754716, 0.769312922748309, 0.8113207547169812, 0.6226415094339622)

## Stacked LSTM

In [195]:
import torch
import torch.nn as nn

class StackedLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(StackedLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # LSTM output: (batch_size, seq_len, hidden_size)
        out = self.fc(lstm_out[:, -1, :])  # Take the last time step's output
        return self.sigmoid(out)


In [196]:
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, confusion_matrix
import numpy as np

# Function to calculate specificity
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

import torch.optim as optim
import time

# Hyperparameters
input_size = 128  # Feature size (e.g., one-hot encoding or embedding)
hidden_size = 128  # LSTM hidden state size
num_layers = 3  # Stacked LSTM layers
output_size = 1  # Binary classification
batch_size = 16
num_epochs = 26
learning_rate = 0.0001
tensorboard_log_dir = f"runs/stacked_lstm/experiment_{time.strftime('%Y-%m-%d_%H-%M-%S')}"

# Model, Loss, Optimizer
model = StackedLSTM(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train and Evaluate
train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, tensorboard_log_dir, num_epochs)
evaluate_model(model, test_loader)


Epoch 1/26, TrainLoss: 0.6947, ValLoss: 0.6935, Accuracy: 0.5000, AUC: 0.6588, Sensitivity: 0.0000, Specificity: 1.0000
Epoch 6/26, TrainLoss: 0.6905, ValLoss: 0.6899, Accuracy: 0.5667, AUC: 0.6607, Sensitivity: 0.7111, Specificity: 0.4222
Epoch 11/26, TrainLoss: 0.6534, ValLoss: 0.6659, Accuracy: 0.5000, AUC: 0.6553, Sensitivity: 0.5556, Specificity: 0.4444
Epoch 16/26, TrainLoss: 0.6289, ValLoss: 0.6382, Accuracy: 0.6000, AUC: 0.6879, Sensitivity: 0.5556, Specificity: 0.6444
Epoch 21/26, TrainLoss: 0.6009, ValLoss: 0.5974, Accuracy: 0.7444, AUC: 0.7546, Sensitivity: 0.6667, Specificity: 0.8222
Epoch 26/26, TrainLoss: 0.5393, ValLoss: 0.5843, Accuracy: 0.7222, AUC: 0.7664, Sensitivity: 0.6222, Specificity: 0.8222
0.7264150943396226
0.7689569241723033
Test Accuracy: 0.7264
Test AUC: 0.7690
Test Sensitivity (Recall): 0.7170
Test Specificity: 0.7358


(0.7264150943396226,
 0.7689569241723033,
 0.7169811320754716,
 0.7358490566037735)

# Generative model

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split

# Define One-Hot Encoding Function for Sequences in PyTorch
def one_hot_torch(seq: str, dtype=torch.float32):
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"  # 20 standard amino acids
    seq_bytes = torch.ByteTensor(list(bytes(seq, "utf-8")))
    aa_bytes = torch.ByteTensor(list(bytes(amino_acids, "utf-8")))
    arr = torch.zeros(len(amino_acids), len(seq_bytes), dtype=dtype)  # One-hot encoded matrix
    for i, aa in enumerate(aa_bytes):
        arr[i, seq_bytes == aa] = 1
    return arr

def preprocess_sequence(full_sequence, max_length):
    """
    Prepares input and target sequences while avoiding padding ('X') during training.
    
    Args:
        full_sequence (str): The original sequence including 'X' padding.
        max_length (int): Maximum sequence length (for padding).
    
    Returns:
        Tuple of (input sequence, target sequence) after removing padding.
    """
    # Remove trailing 'X' (padding) before slicing
    trimmed_sequence = full_sequence.rstrip('X')

    # Ensure there's at least 2 valid residues left for training
    if len(trimmed_sequence) < 2:
        return None, None  # Skip sequences that are too short to predict anything

    # Create input & target sequences (without padding)
    input_seq = trimmed_sequence[:-1]  # Exclude last valid residue
    target_seq = trimmed_sequence[-1]  # Shifted by one position

    # Pad back to maintain fixed-length format
    input_seq = input_seq.ljust(max_length - 1, 'X')  # Pad to max_length - 1
    # target_seq = target_seq.ljust(max_length - 1, 'X')  # Target also gets 'X' padding

    return input_seq, target_seq

# Custom Dataset for Generative Model
class GenerativeSequenceDataset(Dataset):
    def __init__(self, sequences, max_length, one_hot_dtype=torch.float32):
        self.sequences = sequences
        self.max_length = max_length
        self.one_hot_dtype = one_hot_dtype

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        full_sequence = self.sequences.iloc[idx]  # Get sequence

        # Process input & target while handling padding correctly
        input_seq, target_seq = preprocess_sequence(full_sequence, self.max_length)
        if input_seq is None or target_seq is None:
            return None  # Skip sequences that are too short

        # Convert to one-hot encoding
        input_one_hot = one_hot_torch(input_seq, dtype=self.one_hot_dtype)
        target_one_hot = one_hot_torch(target_seq, dtype=self.one_hot_dtype)

        return input_one_hot, target_one_hot


# Split into train (70%), validation (15%), test (15%)
X_train_val, X_test = train_test_split(df["Sequences"], test_size=0.15, random_state=42)
X_train, X_val = train_test_split(X_train_val, test_size=0.15, random_state=42)  

# Convert to PyTorch Dataset
train_dataset = GenerativeSequenceDataset(X_train, max_length)
val_dataset = GenerativeSequenceDataset(X_val, max_length)
test_dataset = GenerativeSequenceDataset(X_test, max_length)

# Define DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Display dataset sizes
dataset_sizes = {
    "Train": len(train_dataset),
    "Validation": len(val_dataset),
    "Test": len(test_dataset)-
}
print(dataset_sizes)




{'Train': 508, 'Validation': 90, 'Test': 106}


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import ReduceLROnPlateau
from icecream import ic

def train_lstm(model, train_loader, val_loader, criterion, optimizer, tensorboard_log_dir, num_epochs=25):
    writer = SummaryWriter(tensorboard_log_dir)
    # Initialize the scheduler: reduce LR by factor 0.5 if validation loss doesn't improve for 5 epochs.
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

    for epoch in range(num_epochs):
        model.train()
        train_losses = []

        for input_seq, target_residue in train_loader:
            optimizer.zero_grad()
            outputs = model(input_seq)  # Forward pass
            outputs = outputs.squeeze()

            # Debug prints for output and target shapes
            ic(outputs.shape)  # Expected: (batch_size, num_classes)
            ic(target_residue.shape)  # Expected: (batch_size, 20)

            # Convert one-hot target to class indices
            target_residue = torch.argmax(target_residue, dim=1)  # (batch_size, 20) → (batch_size,)
            target_residue = target_residue.squeeze()
            ic(target_residue.shape)  # Expected: (batch_size,)
            ic(target_residue)

            # Compute loss (CrossEntropyLoss expects class indices as targets)
            loss = criterion(outputs, target_residue)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        avg_train_loss = np.mean(train_losses)
        writer.add_scalar('Loss/Train', avg_train_loss, epoch)

        # Validation Step
        model.eval()
        val_losses = []
        with torch.no_grad():
            for input_seq, target_residue in val_loader:
                outputs = model(input_seq)
                outputs = outputs.squeeze()

                target_residue = torch.argmax(target_residue, dim=1)  # Converts (batch_size, 20) → (batch_size,)
                target_residue = target_residue.squeeze()
                
                loss = criterion(outputs, target_residue)
                val_losses.append(loss.item())

        avg_val_loss = np.mean(val_losses)
        writer.add_scalar('Loss/Val', avg_val_loss, epoch)

        # Update the learning rate scheduler with the current validation loss
        scheduler.step(avg_val_loss)

        if epoch % 5 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, TrainLoss: {avg_train_loss:.4f}, ValLoss: {avg_val_loss:.4f}')

    writer.close()


### sanity check

In [281]:
def one_hot_to_sequence(one_hot_tensor):
    """
    Converts a one-hot encoded tensor back into an amino acid sequence,
    handling fully zero vectors (padding 'X').

    Args:
        one_hot_tensor (torch.Tensor): One-hot encoded tensor of shape (num_amino_acids, seq_length)

    Returns:
        str: Decoded amino acid sequence
    """
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"  # 20 standard amino acids
    seq_length = one_hot_tensor.shape[1]  # Get sequence length
    
    decoded_sequence = ""
    for i in range(seq_length):
        column = one_hot_tensor[:, i]  # Get one-hot vector for residue
        
        if torch.sum(column) == 0:  # If it's fully zero, assume it's padding
            decoded_sequence += "X"
        else:
            amino_acid_index = torch.argmax(column).item()  # Get highest probability index
            decoded_sequence += amino_acids[amino_acid_index]  # Convert index to amino acid
    
    return decoded_sequence

for x, y in train_loader:
    # print("One-Hot Encoded Input:\n", x)
    # print("One-Hot Encoded Target:\n", y)
    
    # Convert back to amino acid sequences
    input_seq_decoded = one_hot_to_sequence(x[0])  # Decode first sample in batch
    target_seq_decoded = one_hot_to_sequence(y[0])  # Decode first target sequence
    
    print("\nDecoded Input Sequence:", input_seq_decoded)
    print("Decoded Target Sequence:", target_seq_decoded)
    print("Input Shape:", x.shape)
    print("Target Shape:", y.shape)
    break  # Print one batch and exit

for x, y in test_loader:
    # print("One-Hot Encoded Input:\n", x)
    # print("One-Hot Encoded Target:\n", y)
    
    # Convert back to amino acid sequences
    input_seq_decoded = one_hot_to_sequence(x[0])  # Decode first sample in batch
    target_seq_decoded = one_hot_to_sequence(y[0])  # Decode first target sequence
    
    print("\nDecoded Input Sequence:", input_seq_decoded)
    print("Decoded Target Sequence:", target_seq_decoded)
    print("Input Shape:", x.shape)
    print("Target Shape:", y.shape)
    break  # Print one batch and exit


Decoded Input Sequence: KWKSFLKTFKSLKKTVLHTLLKAISXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Decoded Target Sequence: S
Input Shape: torch.Size([16, 20, 127])
Target Shape: torch.Size([16, 20, 1])

Decoded Input Sequence: KFHEKHHSHRGXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Decoded Target Sequence: Y
Input Shape: torch.Size([16, 20, 127])
Target Shape:

 torch.Size([16, 20, 1])


### LSTM

In [288]:
import torch
import torch.nn as nn
import torch.optim as optim
ic.enable()
class LSTM_Generative(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout):
        super(LSTM_Generative, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=-1)  # Softmax for probability distribution

    def forward(self, x):
        ic(x.shape)
        lstm_out, _ = self.lstm(x)  # Shape: (batch_size, seq_length, hidden_size)
        out = self.fc(lstm_out)  # Shape: (batch_size, seq_length, output_size)
        return out # Probability distribution over amino acids


# class LSTM_Generative(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, output_size):
#         super(LSTM_Generative, self).__init__()
#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
#         self.fc = nn.Linear(hidden_size, output_size)  # No softmax!

#     def forward(self, x):
#         lstm_out, _ = self.lstm(x)
#         out = self.fc(lstm_out[:, -1, :])  # Predict next residue from last hidden state
#         return out  # Return logits (no softmax)


In [289]:

ic.disable()
# ic.enable()

# Hyperparameters
input_size = 127 #len(amino_acids)  # Number of amino acids (20)
hidden_size = 128  # LSTM hidden state size
num_layers = 2  # Stacked LSTM layers
output_size = 1 #len(amino_acids)  # Predicting one of 20 amino acids
batch_size = 16
num_epochs = 60
learning_rate = 0.0001
dropout = 0.7
tensorboard_log_dir = f"runs/generative_lstm/experiment_{time.strftime('%Y-%m-%d_%H-%M-%S')}"

# Model, Loss, Optimizer
model = LSTM_Generative(input_size, hidden_size, num_layers, output_size, dropout)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the Model
train_lstm(model, train_loader, val_loader, criterion, optimizer, tensorboard_log_dir, num_epochs)


/mnt/storageG1/lwang/miniconda3/envs/new-ml/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/60, TrainLoss: 2.9947, ValLoss: 2.9937
Epoch 6/60, TrainLoss: 2.9760, ValLoss: 2.9761
Epoch 11/60, TrainLoss: 2.9383, ValLoss: 2.9728
Epoch 16/60, TrainLoss: 2.8913, ValLoss: 2.9548
Epoch 21/60, TrainLoss: 2.8255, ValLoss: 2.9013
Epoch 26/60, TrainLoss: 2.7441, ValLoss: 2.8519
Epoch 31/60, TrainLoss: 2.7049, ValLoss: 2.8736
Epoch 36/60, TrainLoss: 2.6580, ValLoss: 2.8560
Epoch 41/60, TrainLoss: 2.6392, ValLoss: 2.8771
Epoch 46/60, TrainLoss: 2.6337, ValLoss: 2.8783
Epoch 51/60, TrainLoss: 2.6264, ValLoss: 2.8761
Epoch 56/60, TrainLoss: 2.6227, ValLoss: 2.8855


In [284]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score
ic.disable()
def evaluate_lstm(model, test_loader, criterion):
    model.eval()  # Set model to evaluation mode
    
    total_loss = 0
    total_accuracy = 0
    total_perplexity = 0
    num_samples = 0

    with torch.no_grad():
        for input_seq, target_seq in test_loader:
            print('input_seq')
            print(one_hot_to_sequence(input_seq[0]))
            outputs = model(input_seq)  # Predictions (batch_size, seq_length, num_classes)
            print('pred_seq')
            print()
            print(outputs[0])
            print(outputs[0].shape)
            print(one_hot_to_sequence(outputs[0]))
            print('target_seq')
            print(one_hot_to_sequence(target_seq[0]))
            print('---------------------------------------------')
            # Compute loss
            target_indices = torch.argmax(target_seq, dim=-1)  # Shape: (batch_size, seq_length
            # ic(target_indices.shape)
            # ic(outputs.shape)
            # Compute loss (CrossEntropy expects target of shape (batch_size, seq_length))
            loss = criterion(outputs.view(-1, outputs.shape[-1]), target_indices.view(-1))
            total_loss += loss.item()

            # Compute accuracy (compare predicted residues with ground truth)
            predicted_indices = torch.argmax(outputs, dim=-1)  # Get highest probability residues
            target_indices = torch.argmax(target_seq, dim=-1)  # Get actual residues
            batch_accuracy = (predicted_indices == target_indices).float().mean().item()
            total_accuracy += batch_accuracy

            # Compute perplexity (exp of cross-entropy loss)
            batch_perplexity = torch.exp(loss).item()
            total_perplexity += batch_perplexity

            num_samples += 1

    # Calculate averages
    avg_loss = total_loss / num_samples
    avg_accuracy = total_accuracy / num_samples
    avg_perplexity = total_perplexity / num_samples

    print(f"Test Loss: {avg_loss:.4f}")
    print(f"Test Accuracy: {avg_accuracy:.4f}")
    print(f"Test Perplexity: {avg_perplexity:.4f}")

    return avg_loss, avg_accuracy, avg_perplexity
# Run evaluation
test_loss, test_accuracy, test_perplexity = evaluate_lstm(model, test_loader, criterion)
print('test_loss:',test_loss)
print('test_accuracy:',test_accuracy)
print('test_perplexity:',test_perplexity)

input_seq
KFHEKHHSHRGXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
pred_seq

tensor([[ -0.2576],
        [ -1.4460],
        [  0.6699],
        [ -0.1802],
        [ -0.3858],
        [  0.7994],
        [ -2.3871],
        [ -3.3349],
        [ -8.6021],
        [ -5.5967],
        [ -4.2079],
        [ -9.3004],
        [-12.7328],
        [-13.1089],
        [-12.1663],
        [-10.3028],
        [ -7.9509],
        [ -4.1972],
        [ -1.0595],
        [ -8.0496]])
torch.Size([20, 1])
G
target_seq
Y
---------------------------------------------


input_seq
LVRAYHAMXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
pred_seq

tensor([[-0.3253],
        [-4.0561],
        [-2.2803],
        [-1.1629],
        [-0.7106],
        [-0.0308],
        [-1.2208],
        [-2.3512],
        [-5.0872],
        [-4.9262],
        [-2.7832],
        [-8.7870],
        [-7.9234],
        [-4.7137],
        [-3.1058],
        [-4.7313],
        [-5.1510],
        [-3.3170],
        [-4.6499],
        [-3.5734]])
torch.Size([20, 1])
G
target_seq
S
---------------------------------------------
input_seq
AVEAXKTYVRDKPXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
pred_seq

tensor([[  0.0355],
        [ -3.2529],
        [ -0.8319],
        [ -4.1709],
        [-11.3265],
        [ -5.1903],
        [  0.3549],
        [ -0.5790],
        [ -4.1414],
        [ -6.5417],
        [ -8.0048],
        [ -7.5452],
  